In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
df = pd.read_csv('PeliculasCreditos.csv',parse_dates=['release_date'])

Desarrollo API: Propones disponibilizar los datos de la empresa usando el framework FastAPI. Las consultas que propones son las siguientes:

Deben crear 6 funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

-   def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

In [3]:
def cantidad_filmaciones_mes(df, mes):
    # Mapeamos los nombres de meses en español a inglés
    meses_espanol = {
        'enero': 'January',
        'febrero': 'February',
        'marzo': 'March',
        'abril': 'April',
        'mayo': 'May',
        'junio': 'June',
        'julio': 'July',
        'agosto': 'August',
        'septiembre': 'September',
        'octubre': 'October',
        'noviembre': 'November',
        'diciembre': 'December'
    }
    # Convertimos el mes a número en inglés
    mes_numero = pd.to_datetime(meses_espanol[mes.lower()], format='%B').month
    cantidad = sum(df['release_date'].dt.month == mes_numero)    
    return cantidad

In [4]:
cantidad_filmaciones_mes(df,'diciembre')

3781

-   def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

In [5]:
def cantidad_filmaciones_dia(df, dia):
    # Mapeo de nombres de días en español a inglés
    dias_espanol = {
        'lunes': 'Monday',
        'martes': 'Tuesday',
        'miércoles': 'Wednesday',
        'jueves': 'Thursday',
        'viernes': 'Friday',
        'sábado': 'Saturday',
        'domingo': 'Sunday'
    }

    # Convertir el día a nombre en inglés
    dia_ingles = dias_espanol[dia.lower()]

    # Filtrar el DataFrame por el día de estreno
    cantidad = sum(df['release_date'].dt.strftime('%A') == dia_ingles)

    # Devolver las películas del día
    return cantidad

-   def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
                    Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X

In [6]:
def score_titulo(df, titulo):
    pelicula = df[df['title'] == titulo]
    
    anio_estreno = pelicula['release_year'].values[0]

    score = pelicula['popularity'].values[0]

    return titulo, anio_estreno, score

In [7]:
score_titulo(df,'Toy Story')

('Toy Story', 1995, 21.946943)

-   def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

In [8]:
def votos_titulo(df, titulo):
    pelicula = df[df['title'] == titulo]

    if len(pelicula) == 0:
        print("No se encontró ninguna película con ese título.")
        return

    votos = pelicula['vote_count'].values[0]
    promedio_votos = pelicula['vote_average'].values[0]

    if votos < 2000:
        print("La película no cumple con la condición mínima de 2000 valoraciones.")
        return

    return titulo, votos, promedio_votos

In [9]:
votos_titulo(df,'Toy Story')

('Toy Story', 5415.0, 7.7)

-   def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

In [12]:
def get_actor(df, nombre_actor):
    actor = df[df['actores'].apply(lambda x: isinstance(x, str) and nombre_actor in x)]
    
    if len(actor) == 0:
        print("El actor no se encuentra en la lista de actores.")
        return
    
    cantidad_pelis = len(actor)
    retorno_total = actor['return'].sum()
    promedio_retorno = retorno_total / cantidad_pelis

    return nombre_actor, cantidad_pelis, retorno_total, promedio_retorno

In [13]:
get_actor(df,'Joe Johnston')

('Joe Johnston', 3, 70.49072790909091, 23.496909303030304)

-   def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [14]:
def get_director(df, nombre_director):
    director = df[df['Director'] == nombre_director]

    if len(director) == 0:
        print("El director no se encuentra en el dataset.")
        return

    exito_director = director['return'].sum()
    
    peliculas_director = director[['title', 'release_date', 'return', 'budget','revenue']]

    return exito_director, peliculas_director

In [15]:
get_director(df, 'Quentin Tarantino')

(66.72252920242425,
                            title release_date     return     budget    revenue
 292                 Pulp Fiction   1994-09-10  26.741095    8000000  213928762
 1055              Reservoir Dogs   1992-09-02  12.217506    1200000   14661007
 1644                Jackie Brown   1997-12-24   3.306097   12000000   39673162
 6719           Kill Bill: Vol. 1   2003-10-10   6.031633   30000000  180949000
 7265           Kill Bill: Vol. 2   2004-04-16   5.071982   30000000  152159461
 11861                Death Proof   2007-05-21   1.001516   25000000   25037897
 13595       Inglourious Basterds   2009-08-18   4.559015   70000000  319131050
 20025           Django Unchained   2012-12-25   4.253682  100000000  425368238
 28082          The Hateful Eight   2015-12-25   3.540003   44000000  155760117
 41148  My Best Friend's Birthday   1987-01-01   0.000000          0          0)